<a href="https://colab.research.google.com/github/kundajelab/dragonn/blob/master/paper_supplement/LowMemQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#uncomment the lines below if you are running this tutorial from Google Colab 
!pip install dragonn>=0.2.6



torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
seqdataloader 0.121 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.


In [0]:
# Making sure our results are reproducible
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

In [3]:
#load dragonn tutorial utilities 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


from dragonn.tutorial_utils import *

Using TensorFlow backend.


## Input data <a name='1'>
<a href=#outline>Home</a>

Tutorials 1 - 3 have used simulated data generated with the simdna package. In this tutorial, we will examine how well CNN's are able to predict transcription factor binding for four TF's in vivo. 

We will learn to predict transcription factor binding for four transcription factors in the GM12878 cell line (one of the Tier 1 cell lines for the ENCODE project). First, we download the narrowPeak bed files for each of these transcription factors. You can skip the following code block if you already have the data downloaded. 

In [17]:
## SPI1, optimal IDR thresholded peaks, Myers lab, hg19
# https://www.encodeproject.org/experiments/ENCSR000BGQ/
#!wget -O SPI1.narrowPeak.gz http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.narrowPeak.gz
    
## Download "ambiguous" peak sets -- these peaks are in the optimal overlap set across replicates, but are not
## found to be reproducible at a high confidence (p<0.05) by IDR 
#! wget -O SPI1.ambiguous.gz http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.ambiguous.gz
    
## Download the hg19 chromsizes file (We only use chroms 1 -22, X, Y for training)
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.chrom.sizes
    
## Download the hg19 fasta reference genome (and corresponding .fai index)
#!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.genome.fa.gz
#!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.genome.fa.fai 


--2019-02-28 08:29:34--  http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.chrom.sizes
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365
Saving to: ‘hg19.chrom.sizes’

hg19.chrom.sizes    100%[===================>]     365  --.-KB/s    in 0s      

2019-02-28 08:29:34 (58.6 MB/s) - ‘hg19.chrom.sizes’ saved [365/365]



## Generating positive and negative bins for genome-wide training <a name='2'>
<a href=#outline>Home</a>

We will use the *genomewide_labels* function from the  [seqdataloader](https://github.com/kundajelab/seqdataloader) package to generate positive and negative labels for the TF-ChIPseq peaks across the genome. We will treat each sample as a task for the model and compare the performance of the model on SPI1 task in the single-tasked and multi-tasked setting.

In [0]:
from seqdataloader import * 

In [14]:
## seqdataloader accepts an input file, which we call tasks.tsv, with task names in column 1, the corresponding
## peak files in column 2, skip column 3 (which will be used for regression in Tutorial 5), and ambiguous peaks in 
## column4 
with open("tasks.tsv",'w') as f: 
  f.write("\t".join(["SPI1","SPI1.narrowPeak.gz","","SPI1.ambiguous.gz"])+'\n')

! cat tasks.tsv

SPI1	SPI1.narrowPeak.gz		SPI1.ambiguous.gz


With the parameter configuration below, seqdataloader splits the genome into 1kb regions, with a stride of 50. Each 1kb region is centered at a 200 bp bin, with a left flank of 400 bases and a right flank of 400 bases. 

* Each 200 bp bin is labeled as positive if a narrowPeak summit overlaps with it. 

* The bin is labeled ambiguous (label = -1) and excluded from training if there is some overlap with the narrowPeak, but the peak summit does not lie in that overlap. 

* The bin is labeled negative if there is no overlap with the narrowPeak. 

In [15]:
!pwd

/content


In [0]:

positives_train_set_params={
    'store_positives_only':True,
    'task_list':"tasks.tsv",
    'outf':"positives.TF.train.hdf5",
    'output_type':'hdf5',
    'chrom_sizes':'hg19.chrom.sizes',
    'chroms_to_exclude':['chr1','chr2','chr19'],
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'threads':20,
    'subthreads':2,
    'allow_ambiguous':True,
    'output_hdf5_low_mem':True,
    'labeling_approach':'peak_summit_in_bin_classification'
    }
genomewide_labels(positives_train_set_params)

creating dictionary of bed files and bigwig files for each task:
SPI1
No BigWig file was provided for task:SPI1; Make sure this is intentional
creating chromosome thread pool
launching thread pool
pre-allocated df for chrom:chr21with dimensions:(962578, 4)
pre-allocated df for chrom:chr22with dimensions:(1026072, 4)pre-allocated df for chrom:chr18with dimensions:(1561525, 4)

pre-allocated df for chrom:chr17with dimensions:(1623885, 4)
pre-allocated df for chrom:chr15with dimensions:(2050608, 4)pre-allocated df for chrom:chr16with dimensions:(1807076, 4)

pre-allocated df for chrom:chr20with dimensions:(1260491, 4)


In [0]:
positives_valid_set_params={
    'store_positives_only':True,
    'task_list':"tasks.tsv",
    'outf':"positives.TF.valid.hdf5",
    'output_type':'hdf5',
    'chrom_sizes':'hg19.chrom.sizes',
    'chroms_to_keep':'chr1',
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'threads':20,
    'subthreads':2,
    'allow_ambiguous':True,
    'output_hdf5_low_mem':True,
    'labeling_approach':'peak_summit_in_bin_classification'
    }
genomewide_labels(positives_valid_set_params)


In [0]:
positives_test_set_params={
    'store_positives_only':True,
    'task_list':"tasks.tsv",
    'outf':"positives.TF.test.hdf5",
    'output_type':'hdf5',
    'chrom_sizes':'hg19.chrom.sizes',
    'chroms_to_keep':['chr2','chr19'],
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'threads':20,
    'subthreads':2,
    'allow_ambiguous':True,
    'output_hdf5_low_mem':True,
    'labeling_approach':'peak_summit_in_bin_classification'
    }
genomewide_labels(positives_test_set_params)


In [0]:
# When provided with the --store-positives_only flag, the code generates all bins for each task that are labeled positive.
pd.read_hdf("SPI1.positives.TF.train.hdf5",start=0,stop=10)

In [0]:
from dragonn.generators import * 

In [0]:
#To prepare for model training, we import the necessary functions and submodules from keras
from keras.models import Sequential
from keras.layers.core import Dropout, Reshape, Dense, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adadelta, SGD, RMSprop;
import keras.losses;
from keras.constraints import maxnorm;
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping, History, TensorBoard 
from keras import backend as K 
K.set_image_data_format('channels_last')

#we use a custom binary cross-entropy loss that can handle ambiguous labels (denoted with -1 ) and exclude them 
# from the loss calculation 
from dragonn.custom_losses import get_ambig_binary_crossentropy

In [0]:
from concise.metrics import tpr, tnr, fpr, fnr, precision, f1
from keras.constraints import max_norm

def initialize_model(ntasks=1):
    #Define the model architecture in keras (regularized, 3-layer convolution model followed by 1 dense layer)
    model=Sequential() 
    
    model.add(Conv2D(filters=50,kernel_size=(1,15),padding="same", kernel_constraint=max_norm(7.0,axis=-1),input_shape=(1,1000,4)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=50,kernel_size=(1,15),padding="same"))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=50,kernel_size=(1,13),padding="same"))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(1,40)))
    
    model.add(Flatten())
    model.add(Dense(50))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(ntasks))
    model.add(Activation("sigmoid"))
    
    #use the custom ambig_binary_crossentropy loss, indicating that a value of -1 indicates an ambiguous label 
    loss=get_ambig_binary_crossentropy(-1)
    
    ##compile the model, specifying the Adam optimizer, and binary cross-entropy loss. 
    model.compile(optimizer='adam', loss=loss,
                  metrics=[tpr,
                           tnr,
                           fpr,
                           fnr,
                           precision,
                           f1])
    return model

In [0]:
#create the generators
from dragonn.generators import * 
case1_spi1_train_gen=DataGenerator("SPI1.positives.TF.train.hdf5","hg19.genome.fa.gz",shuffled_ref_negatives=True,upsample=False,batch_size=256)
case1_spi1_valid_gen=DataGenerator("SPI1.positives.TF.valid.hdf5","hg19.genome.fa.gz",shuffled_ref_negatives=True,upsample=False,batch_size=256)
case1_ctcf_train_gen=DataGenerator("CTCF.positives.TF.train.hdf5","hg19.genome.fa.gz",shuffled_ref_negatives=True,upsample=False,batch_size=256)
case1_ctcf_valid_gen=DataGenerator("CTCF.positives.TF.valid.hdf5","hg19.genome.fa.gz",shuffled_ref_negatives=True,upsample=False,batch_size=256)

We now follow the standard protocol we used in tutorials 1 - 3 to train a keras model, with the exception that we use the fit_generator function in keras, rather than the fit function.

In [0]:
callbacks=[EarlyStopping(patience=3,restore_best_weights=True),History()]

In [0]:
#If you are running this notebook in google colab, uncomment the lines below to observe the model's training 
!mkdir logs
%tensorboard --logdir logs 
tensorboard_visualizer=TensorBoard(log_dir="logs", histogram_freq=0, batch_size=500, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
callbacks.append(tensorboard_visualizer)

In [0]:
#Train the SPI1 model 
case1_spi1_model=initialize_model()

## use the keras fit_generator function to train the model with early stopping after 3 epochs 
history_case1_spi1=case1_spi1_model.fit_generator(case1_spi1_train_gen,
                                                  validation_data=case1_spi1_valid_gen,
                                                  epochs=150,
                                                  verbose=1,
                                                  use_multiprocessing=True,
                                                  workers=40,
                                                  max_queue_size=100,
                                                  callbacks=callbacks)

Epoch 1/150
2156/2156 [==============================] - 94s 43ms/step - loss: 0.0321 - sensitivity: 0.9871 - specificity: 0.9902 - fpr: 0.0098 - fnr: 0.0129 - precision: 0.9898 - f1: 0.9883 - val_loss: 0.2462 - val_sensitivity: 0.8540 - val_specificity: 1.0000 - val_fpr: 3.0800e-05 - val_fnr: 0.1460 - val_precision: 1.0000 - val_f1: 0.9190
Epoch 2/150
2156/2156 [==============================] - 89s 41ms/step - loss: 0.0075 - sensitivity: 0.9974 - specificity: 0.9977 - fpr: 0.0023 - fnr: 0.0026 - precision: 0.9977 - f1: 0.9976 - val_loss: 0.0135 - val_sensitivity: 0.9976 - val_specificity: 0.9933 - val_fpr: 0.0067 - val_fnr: 0.0024 - val_precision: 0.9934 - val_f1: 0.9955
Epoch 3/150
2156/2156 [==============================] - 93s 43ms/step - loss: 0.0060 - sensitivity: 0.9979 - specificity: 0.9979 - fpr: 0.0021 - fnr: 0.0021 - precision: 0.9979 - f1: 0.9979 - val_loss: 0.0185 - val_sensitivity: 0.9889 - val_specificity: 0.9997 - val_fpr: 2.7720e-04 - val_fnr: 0.0111 - val_precision:

In [0]:
## Plot the learning curves for SPI1  
from dragonn.tutorial_utils import plot_learning_curve
plot_learning_curve(history_case1_spi1)